# Description

It runs DBSCAN on the `umap` version of the data.

The notebook explores different values for `min_samples` and `eps` (the main parameters of DBSCAN).

# Environment variables

In [1]:
from IPython.display import display

import conf

N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

3

In [2]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

env: MKL_NUM_THREADS=3
env: OPEN_BLAS_NUM_THREADS=3
env: NUMEXPR_NUM_THREADS=3
env: OMP_NUM_THREADS=3


# Modules loading

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

from utils import generate_result_set_name
from clustering.ensembles.utils import generate_ensemble

# Global settings

In [5]:
np.random.seed(0)

In [6]:
CLUSTERING_ATTRIBUTES_TO_SAVE = ["n_clusters"]

# Settings

In [7]:
# these parameter values are taken from the pre-analysis notebook for this clustering method and data version
k_values = np.arange(2, 125 + 1, 1)

eps_range_per_k = {
    k: (0.23, 0.80)
    if k < 5
    else (0.30, 0.80)
    if k < 10
    else (0.40, 2.50)
    if k < 20
    else (0.40, 7.00)
    if k < 50
    else (0.50, 10.00)
    for k in k_values
}

# Data version: umap

## Settings

In [8]:
INPUT_SUBSET = "umap"

In [9]:
INPUT_STEM = "z_score_std-projection-smultixcan-efo_partial-mashr-zscores"

In [10]:
# parameters of the dimentionality reduction steps
DR_OPTIONS = {
    "n_components": 50,
    "metric": "euclidean",
    "n_neighbors": 15,
    "random_state": 0,
}

In [11]:
input_filepath = Path(
    conf.RESULTS["CLUSTERING_NULL_DIR"],
    "data_transformations",
    INPUT_SUBSET,
    generate_result_set_name(
        DR_OPTIONS, prefix=f"{INPUT_SUBSET}-{INPUT_STEM}-", suffix=".pkl"
    ),
).resolve()
display(input_filepath)

assert input_filepath.exists(), "Input file does not exist"

input_filepath_stem = input_filepath.stem
display(input_filepath_stem)

PosixPath('/opt/data/results/clustering/null_sims/data_transformations/umap/umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_50-n_neighbors_15-random_state_0.pkl')

'umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_50-n_neighbors_15-random_state_0'

In [12]:
# output dir for this notebook
RESULTS_DIR = Path(
    conf.RESULTS["CLUSTERING_NULL_DIR"],
    "runs",
    f"{INPUT_SUBSET}-{INPUT_STEM}",
).resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/opt/data/results/clustering/null_sims/runs/umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores')

## Load input file

In [13]:
data = pd.read_pickle(input_filepath)

In [14]:
data.shape

(3752, 50)

In [15]:
data.head()

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5,UMAP6,UMAP7,UMAP8,UMAP9,UMAP10,...,UMAP41,UMAP42,UMAP43,UMAP44,UMAP45,UMAP46,UMAP47,UMAP48,UMAP49,UMAP50
100001_raw-Food_weight,6.554080,5.281644,7.335147,4.784761,6.145352,6.611291,5.023619,4.358376,5.172933,6.731123,...,9.107204,7.465721,8.788528,9.572570,9.679030,1.911065,5.402421,5.005345,2.798605,5.674561
100002_raw-Energy,6.460916,5.505183,7.399599,4.883557,6.300282,6.617465,5.325193,4.325032,5.428046,7.023396,...,9.018374,7.488490,8.812325,9.568748,9.703897,1.830261,5.426361,4.880517,2.736183,5.691994
100003_raw-Protein,6.466805,5.458629,7.368454,4.923854,6.393869,6.636221,5.306796,4.363737,5.470543,7.056087,...,9.005486,7.504950,8.769291,9.523876,9.748778,1.860021,5.457011,4.862948,2.789229,5.680467
100004_raw-Fat,6.564027,5.311611,7.302944,4.877467,6.186177,6.692945,5.133474,4.370428,5.349103,6.923361,...,9.122827,7.386698,8.914087,9.622107,9.642089,1.847337,5.383461,4.924160,2.730383,5.653959
100005_raw-Carbohydrate,6.627501,5.250340,7.249473,4.796991,5.966265,6.625223,4.946440,4.482035,5.224523,6.724992,...,9.149892,7.443833,8.843749,9.623201,9.629378,1.898488,5.410903,4.978440,2.766205,5.652066


In [16]:
assert not data.isna().any().any()

## Clustering

### Generate clusterers

In [17]:
CLUSTERING_OPTIONS = {}

# K_RANGE is the min_samples parameter in DBSCAN (sklearn)
CLUSTERING_OPTIONS["K_RANGE"] = k_values
CLUSTERING_OPTIONS["EPS_RANGE_PER_K"] = eps_range_per_k
CLUSTERING_OPTIONS["EPS_STEP"] = 33
CLUSTERING_OPTIONS["METRIC"] = "euclidean"

display(CLUSTERING_OPTIONS)

{'K_RANGE': array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
         41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
         54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
         67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
         80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
         93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
        106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
        119, 120, 121, 122, 123, 124, 125]),
 'EPS_RANGE_PER_K': {2: (0.23, 0.8),
  3: (0.23, 0.8),
  4: (0.23, 0.8),
  5: (0.3, 0.8),
  6: (0.3, 0.8),
  7: (0.3, 0.8),
  8: (0.3, 0.8),
  9: (0.3, 0.8),
  10: (0.4, 2.5),
  11: (0.4, 2.5),
  12: (0.4, 2.5),
  13: (0.4, 2.5),
  14: (0.4, 2.5),
  15: (0.4, 2.5),
  16: (0.4, 2.5),
  1

In [18]:
CLUSTERERS = {}

idx = 0

for k in CLUSTERING_OPTIONS["K_RANGE"]:
    eps_range = CLUSTERING_OPTIONS["EPS_RANGE_PER_K"][k]
    eps_values = np.linspace(eps_range[0], eps_range[1], CLUSTERING_OPTIONS["EPS_STEP"])

    for eps in eps_values:
        clus = DBSCAN(min_samples=k, eps=eps, metric="precomputed", n_jobs=N_JOBS)

        method_name = type(clus).__name__
        CLUSTERERS[f"{method_name} #{idx}"] = clus

        idx = idx + 1

In [19]:
display(len(CLUSTERERS))

4092

In [20]:
_iter = iter(CLUSTERERS.items())
display(next(_iter))
display(next(_iter))

('DBSCAN #0', DBSCAN(eps=0.23, metric='precomputed', min_samples=2, n_jobs=3))

('DBSCAN #1',
 DBSCAN(eps=0.24781250000000002, metric='precomputed', min_samples=2, n_jobs=3))

In [21]:
clustering_method_name = method_name
display(clustering_method_name)

'DBSCAN'

### Generate ensemble

In [22]:
data_dist = pairwise_distances(data, metric=CLUSTERING_OPTIONS["METRIC"])

In [23]:
data_dist.shape

(3752, 3752)

In [24]:
pd.Series(data_dist.flatten()).describe().apply(str)

count            14077504.0
mean     3.0278100967407227
std      1.1191624402999878
min                     0.0
25%       2.238104999065399
50%      3.3084110021591187
75%       3.960253953933716
max       5.226503849029541
dtype: object

In [25]:
ensemble = generate_ensemble(
    data_dist,
    CLUSTERERS,
    attributes=CLUSTERING_ATTRIBUTES_TO_SAVE,
)

100%|██████████| 4092/4092 [11:14<00:00,  6.07it/s]


In [26]:
ensemble.shape

(288, 3)

In [27]:
ensemble.head()

,clusterer_params,partition,n_clusters
clusterer_id,,,
DBSCAN #0,"{'algorithm': 'auto', 'eps': 0.23, 'leaf_size'...","[nan, 0.0, 0.0, nan, nan, 0.0, 0.0, 0.0, nan, ...",22
DBSCAN #1,"{'algorithm': 'auto', 'eps': 0.247812500000000...","[nan, 0.0, 0.0, nan, nan, 0.0, 0.0, 0.0, nan, ...",17
DBSCAN #2,"{'algorithm': 'auto', 'eps': 0.265625, 'leaf_s...","[nan, 0.0, 0.0, nan, nan, 0.0, 0.0, 0.0, 0.0, ...",12
DBSCAN #3,"{'algorithm': 'auto', 'eps': 0.2834375, 'leaf_...","[nan, 0.0, 0.0, nan, nan, 0.0, 0.0, 0.0, 0.0, ...",7
DBSCAN #4,"{'algorithm': 'auto', 'eps': 0.30125, 'leaf_si...","[0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, 0.0, 0.0, ...",5


In [28]:
ensemble["n_clusters"].value_counts().head()

2    239
8      9
7      9
6      7
4      6
Name: n_clusters, dtype: int64

In [29]:
ensemble_stats = ensemble["n_clusters"].describe()
display(ensemble_stats)

count    288.000000
mean       2.840278
std        2.339371
min        2.000000
25%        2.000000
50%        2.000000
75%        2.000000
max       22.000000
Name: n_clusters, dtype: float64

In [30]:
assert (
    ensemble.shape[0] > 0
), "Ensemble is empty, stopping here (this is not actually an error if running null simulations)"

### Testing

In [31]:
assert ensemble_stats["min"] > 1

In [32]:
assert not ensemble["n_clusters"].isna().any()

In [33]:
# assert ensemble.shape[0] == len(CLUSTERERS)

In [34]:
# all partitions have the right size
assert np.all(
    [part["partition"].shape[0] == data.shape[0] for idx, part in ensemble.iterrows()]
)

In [35]:
# no partition has negative clusters (noisy points)
assert not np.any([(part["partition"] < 0).any() for idx, part in ensemble.iterrows()])

### Save

In [36]:
output_filename = Path(
    RESULTS_DIR,
    generate_result_set_name(
        {},
        #         CLUSTERING_OPTIONS,
        prefix=f"{clustering_method_name}",
        suffix=".pkl",
    ),
).resolve()
display(output_filename)

PosixPath('/opt/data/results/clustering/null_sims/runs/umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores/DBSCAN.pkl')

In [37]:
ensemble.to_pickle(output_filename)